## Optimize Model

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns = ['EIN', 'NAME'], inplace=True)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
application_type_value = application_df['APPLICATION_TYPE'].value_counts()
application_type_value

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_type_value[application_type_value<200].index)


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts.loc[classification_counts > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts<500].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df = pd.get_dummies(application_df)
dummies_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False


In [10]:
# Split our preprocessed data into our features and target arrays
X = dummies_df.drop(['IS_SUCCESSFUL'], axis=1).values
y = dummies_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Attempt 1

## Compile, Train and Evaluate the Model

In [12]:

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn_op1 = tf.keras.models.Sequential()

# First hidden layer
nn_op1.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=44))

# Second hidden layer
nn_op1.add(tf.keras.layers.Dense(units=60, activation='relu'))

# third hidden layer
nn_op1.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn_op1.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_op1.summary()

/Users/emelyzelaya/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 80)             │         3,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 60)             │         4,860 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 30)             │         1,830 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,321 (40.32 KB)

 Trainable params: 10,321 (40.32 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
nn_op1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [14]:
# Train the model

fit_model = nn_op1.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 397us/step - accuracy: 0.7120 - loss: 0.5828
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 852us/step - accuracy: 0.7294 - loss: 0.5540
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 473us/step - accuracy: 0.7273 - loss: 0.5556
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 427us/step - accuracy: 0.7320 - loss: 0.5480
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 402us/step - accuracy: 0.7401 - loss: 0.5427
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 367us/step - accuracy: 0.7345 - loss: 0.5462
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 512us/step - accuracy: 0.7335 - loss: 0.5463
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 391us/step - accuracy: 0.7345 - loss: 0.5482
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 495us/step - accuracy: 0.7372 - loss: 0.5429
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 685us/step - accuracy: 0.7383 - loss: 0.5439
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 445us/step - accuracy: 0.7341 - loss: 0.5432
Epoch 12/100
804/80

In [15]:
# Evaluate the model using the test data
model_loss_op1, model_accuracy_op1 = nn_op1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_op1}, Accuracy: {model_accuracy_op1}")

268/268 - 0s - 423us/step - accuracy: 0.7268 - loss: 0.5680
Loss: 0.5680295825004578, Accuracy: 0.7267638444900513


In [16]:
# Export our model to HDF5 file
nn_op1.save('Results/AlphabetSoupCharity_Optimization1.h5')

Attempt 2

## Compile, Train and Evaluate the Model

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn_op2 = tf.keras.models.Sequential()

# First hidden layer
nn_op2.add(tf.keras.layers.Dense(units=75, activation='relu', input_dim=44))

# Second hidden layer
nn_op2.add(tf.keras.layers.Dense(units=35, activation='relu'))

# Output layer
nn_op2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_op2.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 75)             │         3,375 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 35)             │         2,660 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            36 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,071 (23.71 KB)

 Trainable params: 6,071 (23.71 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
nn_op2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [19]:
fit_model_op2 = nn_op2.fit(X_train_scaled, y_train, epochs=100)


Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 327us/step - accuracy: 0.7029 - loss: 0.5936
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 326us/step - accuracy: 0.7301 - loss: 0.5556
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 352us/step - accuracy: 0.7318 - loss: 0.5498
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 336us/step - accuracy: 0.7315 - loss: 0.5511
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 377us/step - accuracy: 0.7332 - loss: 0.5473
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 332us/step - accuracy: 0.7374 - loss: 0.5416
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 362us/step - accuracy: 0.7348 - loss: 0.5487
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 330us/step - accuracy: 0.7290 - loss: 0.5496
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 325us/step - accuracy: 0.7331 - loss: 0.5469
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 322us/step - accuracy: 0.7350 - loss: 0.5421
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 434us/step - accuracy: 0.7386 - loss: 0.5437
Epoch 12/100
804/80

In [20]:
# Evaluate the model using the test data
model_loss_op2, model_accuracy_op2 = nn_op2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_op2}, Accuracy: {model_accuracy_op2}")

268/268 - 0s - 467us/step - accuracy: 0.7257 - loss: 0.5562
Loss: 0.5562165379524231, Accuracy: 0.7257142663002014


In [21]:
# Export our model to HDF5 file
nn_op2.save('Results/AlphabetSoupCharity_Optimization2.h5')

Attempt 3

## Compile, Train and Evaluate the Model

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn_op3 = tf.keras.models.Sequential()

# First hidden layer
nn_op3.add(tf.keras.layers.Dense(units=120, activation='relu', input_dim=44))

# Second hidden layer
nn_op3.add(tf.keras.layers.Dense(units=75, activation='tanh'))

# third hidden layer
nn_op3.add(tf.keras.layers.Dense(units=35, activation="tanh"))

# Output layer
nn_op3.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_op3.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                 │ (None, 120)            │         5,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 75)             │         9,075 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 35)             │         2,660 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │            36 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,171 (67.07 KB)

 Trainable params: 17,171 (67.07 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:

# Compile the model
nn_op3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
     

In [24]:

# Train the model

fit_model_op3 = nn_op3.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 411us/step - accuracy: 0.7142 - loss: 0.5786
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 399us/step - accuracy: 0.7281 - loss: 0.5563
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 399us/step - accuracy: 0.7299 - loss: 0.5516
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 401us/step - accuracy: 0.7290 - loss: 0.5506
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 399us/step - accuracy: 0.7316 - loss: 0.5470
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 506us/step - accuracy: 0.7335 - loss: 0.5484
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 492us/step - accuracy: 0.7376 - loss: 0.5429
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 418us/step - accuracy: 0.7372 - loss: 0.5452
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 415us/step - accuracy: 0.7314 - loss: 0.5491
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 448us/step - accuracy: 0.7336 - loss: 0.5450
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 407us/step - accuracy: 0.7335 - loss: 0.5474
Epoch 12/100
804/80

In [25]:
# Evaluate the model using the test data
model_loss_op3, model_accuracy_op3 = nn_op3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_op3}, Accuracy: {model_accuracy_op3}")

268/268 - 0s - 531us/step - accuracy: 0.7250 - loss: 0.5549
Loss: 0.5548619627952576, Accuracy: 0.7250145673751831


In [26]:

# Export our model to HDF5 file
nn_op3.save('Results/AlphabetSoupCharity_Optimization3.h5')

All Attempts failed. I couldn't achieve a target predictive accuracy higher than 75%.